In [1]:
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import transforms
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torch.optim import SGD
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
#from engine import train_one_epoch, evaluate

import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
import json
import PIL.Image
from matplotlib import pyplot as plt



In [3]:
from face_mask_datasets import MaskDetectionDatasetJSON, MaskDetectionDatasetXML

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

CLASS_LABELS_ALL = {
    "hijab_niqab": 0,
    "mask_colorful": 1,
    "mask_surgical": 2,
    "face_no_mask": 3,
    "face_with_mask_incorrect": 4,
    "face_with_mask": 5,
    "face_other_covering": 6,
    "scarf_bandana": 7,
    "balaclava_ski_mask": 8,
    "face_shield": 9,
    "other": 10,
    "gas_mask": 11,
    "turban": 12,
    "helmet": 13,
    "sunglasses": 14,
    "eyeglasses": 15,
    "hair_net": 16,
    "hat": 17,
    "goggles": 18,
    "hood": 19
}

CLASS_LABELS_FACES = {
    "face_no_mask": 0,
    "face_with_mask_incorrect": 1,
    "face_with_mask": 2,
    "face_other_covering": 3
}

TRAIN_PRECENTAGE = 0.8  # 80% of the data is used for training
BATCH_SIZE = 12
NUM_EPOCHS = 1
LEARNING_RATE = 0.005
print("Using device: ", device)

Using device:  cuda


In [4]:
# dataset_big = MaskDetectionDatasetJSON("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask",CLASS_LABELS, (512,512))
# dataset_big_singleFaces = MaskDetectionDatasetJSON("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask",CLASS_LABELS,(512,512), only_single_faces=True)
# dataset_big_multipleFaces = MaskDetectionDatasetJSON("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask",CLASS_LABELS,(512,512), only_multiple_faces=True)

# dataset_small = MaskDetectionDatasetXML("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Kaggle Face Mask Detection Full",(512,512))
# dataset_small_low_light = MaskDetectionDatasetXML("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Kaggle Face Mask Detection Full",(512,512), use_dark_images=True)

# #create a dict of the datasets

# datasets = {
#     "dataset_big": dataset_big,
#     "dataset_big_singleFaces": dataset_big_singleFaces,
#     "dataset_big_multipleFaces": dataset_big_multipleFaces,
#     "dataset_small": dataset_small,
#     "dataset_small_low_light": dataset_small_low_light
#}

In [5]:
dataset_big_all_classes = MaskDetectionDatasetJSON("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask",CLASS_LABELS_ALL, (512,512), allowed_classes=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19])
dataset_big_face_classes = MaskDetectionDatasetJSON("C:\GIT_Workspaces\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask",CLASS_LABELS_FACES,(512,512), allowed_classes=[0,1,2,3])

#create a dict of the datasets

datasets = {
    "dataset_big_all_classes": dataset_big_all_classes,
    "dataset_big_face_classes": dataset_big_face_classes
}

c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 1855.jpg has no annotations
  warnings.warn(f"File {file_name} has no annotations")
c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 1945.jpg has no annotations
  warnings.warn(f"File {file_name} has no annotations")
c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 2135.jpg has no annotations
  warnings.warn(f"File {file_name} has no annotations")
c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 3102.png has no annotations
  warnings.warn(f"File {file_name} has no annotations")
c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 4618.png has no annotations
  warnings.warn(f"File {file_name} has no annotations")
c:\GIT_Workspaces\Deep-Vision-sta\Faster R-CNN\face_mask_datasets.py:59: UserWarning: File 5012.jpg has no annotations
  warn

In [6]:
dataset_big_face_classes.annotations[52]

([{'isProtected': False,
   'ID': 537667945160657984,
   'BoundingBox': [680, 340, 1664, 1784],
   'classname': 'face_no_mask',
   'Confidence': 1,
   'Attributes': {}}],
 '1856.jpg')

In [7]:
dataset_big_all_classes.annotations[52]

([{'isProtected': False,
   'ID': 702780802691515392,
   'BoundingBox': [480, 143, 586, 230],
   'classname': 'mask_surgical',
   'Confidence': 1,
   'Attributes': {}}],
 '1855.jpg')

In [8]:
dataset_big_face_classes.__len__()

4314

In [9]:
#create dataloaders for datasets

import torch
from torch.utils.data import Dataset, DataLoader
import utils

dataloaders = {}

for dataset_name, dataset in datasets.items():
    train_size = int(TRAIN_PRECENTAGE * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=utils.collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=utils.collate_fn)
    dataloaders[dataset_name] = {
        "train": train_dataloader,
        "test": test_dataloader
    }

In [10]:
model_resnet_all_classes = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model_resnet_face_classes = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model_mobile_all_classes = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
model_mobile_face_classes = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

models_classComparison = {
    "resnet":{
        "all_classes": model_resnet_all_classes,
        "face_classes": model_resnet_face_classes
    },
    "mobile":{
        "all_classes": model_mobile_all_classes,
        "face_classes": model_mobile_face_classes
    }
}

num_classes = {
    "all_classes": 20,
    "face_classes": 4
}

#prepare models

for model_name, model_dict in models_classComparison.items():
    for model_type, model in model_dict.items():
        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features

        # number of classes
        nc = num_classes[model_type]

        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, nc)

        # move model to the right device
        model.to(device)

c:\Users\johan\.conda\envs\DV-STA\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\johan\.conda\envs\DV-STA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\johan\.conda\envs\DV-STA\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1`. You 

In [11]:
classnames = set()

for item in dataset_big_face_classes.annotations:
    objects, _ = item
    for obj in objects:
        classnames.add(obj['classname'])

unique_classnames = list(classnames)
print(unique_classnames)

['face_with_mask_incorrect', 'face_with_mask', 'face_no_mask', 'face_other_covering']


In [12]:
import copy
from engine import train_one_epoch, evaluate

def train_model(model, dataloaders, optimizer, num_epochs=10):

    # Define empty arrays to collect metrics
    ap_values = []
    ar_values = []

    losses = []
    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, dataloaders["train"], device, epoch, print_freq=1, losses_out=losses)
        # evaluate on the test dataset
        evaluator = evaluate(model, dataloaders["test"], device)

        # Extract the metrics from the evaluator
        average_precisions = evaluator.coco_eval['bbox'].stats[:6]
        average_recalls = evaluator.coco_eval['bbox'].stats[6:]

        # Append the metrics to the arrays
        ap_values.append(average_precisions)
        ar_values.append(average_recalls)

    return model, ap_values, ar_values, losses

In [13]:
#train_model(model_resnet_all_classes, dataloaders["dataset_big_all_classes"], SGD(model_resnet_all_classes.parameters(), lr=LEARNING_RATE), num_epochs=NUM_EPOCHS)

In [14]:
datasets["dataset_big_all_classes"].__len__()

4326

In [15]:
dataset_big_all_classes.__len__()

4326

In [16]:
dataset_big_face_classes.__len__()

4314

In [17]:
datasets["dataset_big_face_classes"].__len__()

4314

In [18]:
train_model(model_resnet_face_classes, dataloaders["dataset_big_face_classes"], SGD(model_resnet_face_classes.parameters(), lr=LEARNING_RATE), num_epochs=NUM_EPOCHS)

Epoch: [0]  [  0/288]  eta: 0:16:15  lr: 0.000022  loss: 1.7590 (1.7590)  loss_classifier: 1.5279 (1.5279)  loss_box_reg: 0.1420 (0.1420)  loss_objectness: 0.0427 (0.0427)  loss_rpn_box_reg: 0.0464 (0.0464)  time: 3.3860  data: 0.2070  max mem: 8869
Epoch: [0]  [  1/288]  eta: 0:10:05  lr: 0.000040  loss: 1.7590 (1.7658)  loss_classifier: 1.5279 (1.5290)  loss_box_reg: 0.1420 (0.1619)  loss_objectness: 0.0427 (0.0468)  loss_rpn_box_reg: 0.0098 (0.0281)  time: 2.1110  data: 0.1600  max mem: 8870
Epoch: [0]  [  2/288]  eta: 0:07:53  lr: 0.000057  loss: 1.7725 (1.7796)  loss_classifier: 1.5279 (1.5235)  loss_box_reg: 0.1818 (0.1761)  loss_objectness: 0.0509 (0.0548)  loss_rpn_box_reg: 0.0194 (0.0252)  time: 1.6540  data: 0.1453  max mem: 8870
Epoch: [0]  [  3/288]  eta: 0:06:59  lr: 0.000075  loss: 1.7590 (1.7713)  loss_classifier: 1.5279 (1.5282)  loss_box_reg: 0.1609 (0.1723)  loss_objectness: 0.0427 (0.0491)  loss_rpn_box_reg: 0.0112 (0.0217)  time: 1.4720  data: 0.1645  max mem: 8870


Epoch: [0]  [ 44/288]  eta: 0:03:25  lr: 0.000788  loss: 0.5784 (1.0578)  loss_classifier: 0.3830 (0.8628)  loss_box_reg: 0.1302 (0.1366)  loss_objectness: 0.0393 (0.0482)  loss_rpn_box_reg: 0.0090 (0.0102)  time: 0.7683  data: 0.1625  max mem: 8870


In [ ]:
#train the models on the big dataset with all classes and single faces
import numpy as np
import os

models_trained = {}

model_type__dataloader_map = {
    "all_classes": "dataset_big_all_classes",
    "face_classes": "dataset_big_face_classes"
}


for model_name, model_dict in models_classComparison.items():
    models_trained[model_name] = {}
    for model_type, model in model_dict.items():
        
        print("Training model: ", model_name, " ", model_type)
        print("using dataset: ", model_type__dataloader_map[model_type])
        print("--------------------------------------------------")
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=0.9, weight_decay=0.0005)
        model, ap_values, ar_values, losses = train_model(model, dataloaders[model_type__dataloader_map[model_type]], optimizer, num_epochs=NUM_EPOCHS)
        models_trained[model_name][model_type] = model

        # Save the model into the "finished_models" folder
        model_filename = f"{model_name}_{model_type}.pth"
        model_path = os.path.join("finished_models/class_compare/", model_filename)
        torch.save(model.state_dict(), model_path)

        # Save the metrics into the "metric_history" folder
        metric_filename = f"{model_name}_{model_type}_metrics.npz"
        metric_path = os.path.join("finished_models/class_compare/metric_history", metric_filename)
        np.savez(metric_path, ap_values=ap_values, ar_values=ar_values, losses=losses)
